##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pesquisador de texto com o TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_searcher"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/google/universal-sentence-encoder-lite/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

Neste notebook do Colab, você aprenderá a usar a biblioteca [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TensorFlow Lite) para criar um modelo de pesquisador do TF Lite. Você pode usar um modelo de pesquisador de texto para criar uma pesquisa semântica ou resposta inteligente para seu aplicativo. Esse tipo de modelo permite receber uma consulta de texto e pesquisar as entradas mais relacionadas em um dataset de texto, como um banco de dados de páginas web. O modelo retorna uma lista das entradas de pontuação com a menor distância no dataset, incluindo os metadados que você especificar, como URL, título da página ou outros identificadores de entrada de texto. Após compilar o modelo, você pode implantá-lo em dispositivos (como Android) usando a [API Task Library Searcher](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher) para executar a inferência com apenas algumas linhas de código.

Este tutorial usa o dataset CNN/DailyMail como uma instância para criar o modelo de pesquisador do TF Lite. Você pode usar seu próprio dataset no formato de valor separado por vírgula (CSV).

## Pesquisa de texto usando Scalable Nearest Neighbor (vizinho mais próximo escalável)

Este tutorial usa o dataset de resumo não anonimizado CNN/DailyMail, disponível publicamente, que foi gerado pelo [repositório do GitHub](https://github.com/abisee/cnn-dailymail). Esse dataset contém mais de 300 mil notícias e, portanto, é um bom dataset para criar o modelo de pesquisador, além de retornar diversas notícias relacionadas durante a inferência do modelo dada uma consulta de texto.

O modelo de pesquisador de texto deste exemplo usa um arquivo de índice de [ScaNN](https://github.com/google-research/google-research/tree/master/scann) (Scalable Nearest Neighbors) que pode pesquisar itens similares em um banco de dados predefinido. O ScaNN alcança um desempenho excepcional em pesquisa eficiente de similaridade de vetores em grande escala.

Os destaques e as URLs nesse dataset são usados neste Colab para criar o modelo:

1. Os destaques são os textos para gerar os vetores de características de embedding, e depois são usados para fazer a pesquisa.
2. As URLs são os resultados retornados exibidos para os usuários após procurar os destaques relacionados.

Este tutorial salva esses dados no arquivo CSV e depois usa esse arquivo para compilar o modelo. Veja diversos exemplos do dataset.

Destaques | URLs
--- | ---
Hawaiian Airlines again lands at No. 1 in on-time performance. The Airline Quality Rankings Report looks at the 14 largest U.S. airlines. ExpressJet <br> and American Airlines had the worst on-time performance. Virgin America had the best baggage  handling; Southwest had lowest complaint rate. (A Hawaiian Airlines figura novamente em 1º lugar em pontualidade. O Relatório de Classificação de Qualidade das Linhas Aéreas avalia as 14 maiores linhas aéreas dos EUA. A ExpressJet e a American Airlines tiveram a pior pontualidade. A Virgin America teve o melhor manuseio de bagagens; a Southwest teve a taxa de reclamações mais baixa.) | http://www.cnn.com/2013/04/08/travel/airline-quality-report
European football's governing body reveals list of countries bidding to host 2020 finals. The 60th anniversary edition of the finals will be hosted by 13 <br> countries. Thirty-two countries are considering bids to host 2020 matches. UEFA will announce host cities on September 25. (A agência reguladora de futebol europeia revela a lista de países dando lances para sediar as finais de 2020. A edição de 60º aniversário das finais será realizada por 13 países. Trinta e dois países consideram dar lances para sediar as partidas de 2020. A UEFA anunciará as cidades-sede em 25 de setembro.) | http://edition.cnn.com:80/2013/09/20/sport/football/football-euro-2020-bid-countries/index.html?
Once octopus-hunter Dylan Mayer has now also signed a petition of 5,000 divers banning their hunt at Seacrest Park. Decision by Washington <br> Department of Fish and Wildlife could take months. (O ex-caçador de polvos Dylan Mayer assinou uma petição de 5.000 mergulhadores banindo a caçada em Seacrest Park. A decisão do Departamento de Pesca e Vida Selvagem de Washington poderá demorar meses.) | http://www.dailymail.co.uk:80/news/article-2238423/Dylan-Mayer-Washington-considers-ban-Octopus-hunting-diver-caught-ate-Puget-Sound.html?
Galaxy was observed 420 million years after the Big Bang. found by NASA’s Hubble Space Telescope, Spitzer Space Telescope, and one of nature’s <br> own natural 'zoom lenses' in space. (A galáxia foi observada 420 milhões de anos após o Big Bang. Foi encontrada pelo telescópio espacial Hubble da NASA, pelo telescópio espacial Spitzer e por uma das "lentes de zoom" naturais da natureza no espaço.) | http://www.dailymail.co.uk/sciencetech/article-2233883/The-furthest-object-seen-Record-breaking-image-shows-galaxy-13-3-BILLION-light-years-Earth.html


## Configuração


Comece instalando os pacotes obrigatórios, incluindo o pacote do Model Maker disponível no [repositório do GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!pip install gdown

Importe os pacotes necessários.

In [ ]:
from tflite_model_maker import searcher

### Prepare o dataset

Este tutorial usa o dataset de resumo CNN/DailyMail do [repositório do GitHub](https://github.com/abisee/cnn-dailymail).

Primeiro baixe os textos e URLs da CNN e DailyMail e descompacte-os. Se houver falha ao baixar do Google Drive, aguarde alguns minutos para tentar novamente ou baixe-os manualmente e carregue-os no Colab.

In [ ]:
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfTHk4NFg2SndKcjQ
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfM1BxdkxVaTY2bWs

!wget -O all_train.txt https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt
!tar xzf cnn_stories.tgz
!tar xzf dailymail_stories.tgz

Em seguida, salve os dados no arquivo CSV, que pode ser carregado na biblioteca `tflite_model_maker`. O código baseia-se na lógica usada para carregar esses dados em [`tensorflow_datasets`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/summarization/cnn_dailymail.py). Não podemos usar `tensorflow_dataset` diretamente, já que ele não contém as URLs que são usadas neste Colab.

Como demora um longo tempo para processar os dados em vetores de características de embedding para todo o dataset, somente os primeiros 5% das histórias do dataset CNN/DailyMail são selecionados por padrão para fins de demonstração. Você também pode ajustar a porcentagem ou usar o [modelo](https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/cnn_daily_text_searcher.tflite) pré-compilado do TF Lite com 50% das histórias do dataset CNN/DailyMail para fazer a pesquisa.

In [ ]:
#@title Save the highlights and urls to the CSV file
#@markdown Load the highlights from the stories of CNN / Daily Mail, map urls with highlights, and save them to the CSV file.

CNN_FRACTION = 0.05 #@param {type:"number"}
DAILYMAIL_FRACTION = 0.05 #@param {type:"number"}

import csv
import hashlib
import os
import tensorflow as tf

dm_single_close_quote = u"\u2019"  # unicode
dm_double_close_quote = u"\u201d"
END_TOKENS = [
    ".", "!", "?", "...", "'", "`", '"', dm_single_close_quote,
    dm_double_close_quote, ")"
]  # acceptable ways to end a sentence


def read_file(file_path):
  """Reads lines in the file."""
  lines = []
  with tf.io.gfile.GFile(file_path, "r") as f:
    for line in f:
      lines.append(line.strip())
  return lines


def url_hash(url):
  """Gets the hash value of the url."""
  h = hashlib.sha1()
  url = url.encode("utf-8")
  h.update(url)
  return h.hexdigest()


def get_url_hashes_dict(urls_path):
  """Gets hashes dict that maps the hash value to the original url in file."""
  urls = read_file(urls_path)
  return {url_hash(url): url[url.find("id_/") + 4:] for url in urls}


def find_files(folder, url_dict):
  """Finds files corresponding to the urls in the folder."""
  all_files = tf.io.gfile.listdir(folder)
  ret_files = []
  for file in all_files:
    # Gets the file name without extension.
    filename = os.path.splitext(os.path.basename(file))[0]
    if filename in url_dict:
      ret_files.append(os.path.join(folder, file))
  return ret_files


def fix_missing_period(line):
  """Adds a period to a line that is missing a period."""
  if "@highlight" in line:
    return line
  if not line:
    return line
  if line[-1] in END_TOKENS:
    return line
  return line + "."


def get_highlights(story_file):
  """Gets highlights from a story file path."""
  lines = read_file(story_file)

  # Put periods on the ends of lines that are missing them
  # (this is a problem in the dataset because many image captions don't end in
  # periods; consequently they end up in the body of the article as run-on
  # sentences)
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  highlight_list = []
  next_is_highlight = False
  for line in lines:
    if not line:
      continue  # empty line
    elif line.startswith("@highlight"):
      next_is_highlight = True
    elif next_is_highlight:
      highlight_list.append(line)

  # Make highlights into a single string.
  highlights = "\n".join(highlight_list)

  return highlights

url_hashes_dict = get_url_hashes_dict("all_train.txt")
cnn_files = find_files("cnn/stories", url_hashes_dict)
dailymail_files = find_files("dailymail/stories", url_hashes_dict)

# The size to be selected.
cnn_size = int(CNN_FRACTION * len(cnn_files))
dailymail_size = int(DAILYMAIL_FRACTION * len(dailymail_files))
print("CNN size: %d"%cnn_size)
print("Daily Mail size: %d"%dailymail_size)

with open("cnn_dailymail.csv", "w") as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=["highlights", "urls"])
  writer.writeheader()

  for file in cnn_files[:cnn_size] + dailymail_files[:dailymail_size]:
    highlights = get_highlights(file)
    # Gets the filename which is the hash value of the url.
    filename = os.path.splitext(os.path.basename(file))[0]
    url = url_hashes_dict[filename]
    writer.writerow({"highlights": highlights, "urls": url})


## Compile o modelo de pesquisador de texto

Para criar um modelo de pesquisador de texto, carregue um dataset, crie um modelo com os dados e exporte o modelo do TF Lite.

### Etapa 1 – Carregue o dataset

O Model Maker recebe o dataset de texto e os metadados correspondentes de cada string de texto (como URLs, neste exemplo) no formato CSV. Ele incorpora as strings de texto aos vetores de características usando o modelo de incorporador especificado pelo usuário.

Nesta demonstração, criamos o modelo de pesquisador usando o [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder-lite/2) (encoder de frases universal), um modelo moderno de embedding de frases que já foi retreinado no [Colab](https://github.com/tensorflow/tflite-support/blob/master/tensorflow_lite_support/examples/colab/on_device_text_to_image_search_tflite.ipynb). O modelo é otimizado para ter alto desempenho de inferência em dispositivos e demora somente 6 ms para incorporar uma string de consulta (essa medida foi feita no Pixel 6). Outra opção é usar [esta](https://tfhub.dev/google/lite-model/universal-sentence-encoder-qa-ondevice/1?lite-format=tflite) versão quantizada, que é menor, mas demora 38 ms para cada embedding.

In [ ]:
!wget -O universal_sentence_encoder.tflite https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/text_embedder.tflite

Crie uma instância de `searcher.TextDataLoader` e use o método `data_loader.load_from_csv` para carregar o dataset. Esta etapa demora cerca de 10 minutos, pois é gerado o vetor de características de embedding para cada texto, um por um. Você também pode enviar seu próprio arquivo CSV e carregá-lo para compilar o modelo personalizado.

Especifique o nome da coluna de texto e a coluna de metadados no arquivo CSV.

- O texto é usado para gerar os vetores de características de embedding.
- Os metadados são o contexto a ser exibido ao pesquisar um determinado texto.

Veja as primeiras quatro linhas do arquivo CSV CNN-DailyMail gerado acima.

Destaques | URLs
--- | ---
Syrian official: Obama climbed to the top of the tree, doesn't know how to get down. Obama sends a letter to the heads of the House and Senate. Obama <br> to seek congressional approval on military action against Syria. Aim is to determine whether CW were used, not by whom, says U.N. spokesman. (Representante da Síria: Obama subiu no topo da árvore e não sabe como descer. Obama envia uma carta aos presidentes da Câmara e do Senado. Obama busca aprovação do Congresso para ação militar contra a Síria. O objetivo é determinar se armas químicas foram usadas, não por quem, diz porta-voz da ONU.) | http://www.cnn.com/2013/08/31/world/meast/syria-civil-war/
Usain Bolt wins third gold of world championship. Anchors Jamaica to 4x100m relay victory. Eighth gold at the championships for Bolt. Jamaica double <br> up in women's 4x100m relay. (Usain Bolt ganha o terceiro ouro no campeonato mundial. Leva a Jamaica à vitória do revezamento 4 x 100 metros. Oitavo ouro em campeonatos para Bolt. Dobradinha da Jamaica com vitória do revezamento 4 x 100 metros feminino.) | http://edition.cnn.com/2013/08/18/sport/athletics-bolt-jamaica-gold
The employee in agency's Kansas City office is among hundreds of "virtual" workers. The employee's travel to and from the mainland U.S. last year cost <br> more than $24,000. The telecommuting program, like all GSA practices, is under review. (O funcionário no gabinete da agência em Kansas City está entre centenas de trabalhadores "virtuais". O deslocamento do funcionário para os EUA continentais e dos EUA continentais custou mais de US$ 24 mil no ano passado. O programa de teletrabalho, assim como todas as práticas da GSA, estão sob análise.) | http://www.cnn.com:80/2012/08/23/politics/gsa-hawaii-teleworking
NEW: A Canadian doctor says she was part of a team examining Harry Burkhart in 2010. NEW: Diagnosis: "autism, severe anxiety, post-traumatic stress <br> disorder and depression" Burkhart is also suspected in a German arson probe, officials say. Prosecutors believe the German national set a string of fires <br> in Los Angeles. (NOVIDADE: Uma médica canadense disse que participou da equipe que examinou Harry Burkhart em 2010. NOVIDADE: Diagnóstico: "autismo, ansiedade grave, transtorno de estresse pós-traumático e depressão". Burkhart também é suspeito em uma investigação alemã de incêndio criminoso, segundo os oficiais. Promotores acreditam que o cidadão alemão causou diversos incêndios em Los Angeles.) | http://edition.cnn.com:80/2012/01/05/justice/california-arson/index.html?


In [ ]:
data_loader = searcher.TextDataLoader.create("universal_sentence_encoder.tflite", l2_normalize=True)
data_loader.load_from_csv("cnn_dailymail.csv", text_column="highlights", metadata_column="urls")

Para casos de uso com imagens, você pode criar uma instância de `searcher.ImageDataLoader` e depois usar `data_loader.load_from_folder` para carregar imagens a partir da pasta. A instância de `searcher.ImageDataLoader` precisa ser criada por um modelo de incorporador do TF Lite, pois ele será usado para codificar consultas em vetores de características e será exportado com o modelo de pesquisador do TF Lite. Por exemplo:

```python
data_loader = searcher.ImageDataLoader.create("mobilenet_v2_035_96_embedder_with_metadata.tflite")
data_loader.load_from_folder("food/")
```

Etapa 2 – Crie o modelo de pesquisador

- Configure as opções do ScaNN. Confira mais detalhes na [documentação da API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/searcher/ScaNNOptions).
- Crie o modelo de pesquisador a partir dos dados e opções do ScaNN. Confira a [análise detalhada](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) para saber mais sobre o algoritmo do ScaNN.

In [ ]:
scann_options = searcher.ScaNNOptions(
      distance_measure="dot_product",
      tree=searcher.Tree(num_leaves=140, num_leaves_to_search=4),
      score_ah=searcher.ScoreAH(dimensions_per_block=1, anisotropic_quantization_threshold=0.2))
model = searcher.Searcher.create_from_data(data_loader, scann_options)

No exemplo acima, definimos as seguintes opções:

- `distance_measure`: usamos o produto escalar para mensurar a distância entre dois vetores de embedding. Observação: nós computamos o valor do produto escalar **negativo** para preservar a noção de que "menor é mais próximo".

- `tree`: o dataset é dividido em 140 partições (aproximadamente, a raiz quadrada do tamanho dos dados), e 4 delas são pesquisadas durante a recuperação, o que corresponde a cerca de 3% do dataset.

- `score_ah`: quantizamos os embeddings float em valores int8 com a mesma dimensão para economizar espaço.

Etapa 3 – Exporte o modelo do TF Lite

Em seguida, você pode exportar o modelo de pesquisador do TF Lite.

In [ ]:
model.export(
      export_filename="searcher.tflite",
      userinfo="",
      export_format=searcher.ExportFormat.TFLITE)

## Teste o modelo do TF Lite com sua pesquisa

Você pode testar o modelo do TF Lite exportado com um texto de consulta personalizado. Para consultar o texto usando o modelo de pesquisador, inicialize o modelo e execute uma pesquisa com a frase da seguinte forma:

In [ ]:
from tflite_support.task import text

# Initializes a TextSearcher object.
searcher = text.TextSearcher.create_from_file("searcher.tflite")

# Searches the input query.
results = searcher.search("The Airline Quality Rankings Report looks at the 14 largest U.S. airlines.")
print(results)

Confira mais informações sobre como integrar o modelo a diversas plataformas na [documentação da Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher).

# Saiba mais

Confira mais informações em:

- [Guia](https://www.tensorflow.org/lite/models/modify/model_maker) e [referência da API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) do TensorFlow Lite Model Maker.

- Task Library: [TextSearcher](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher) para implantação.

- Aplicativos de referência completos para [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_searcher/android).
